In [ ]:
import librosa
# import librosa.display
# import IPython.display as ipd

# import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import cv2
import os

import tensorflow as tf
from tensorflow import keras
from keras import layers, Model
import warnings
warnings.filterwarnings('ignore')

<h3> Video Data Reading </h3>

In [ ]:
video_data = []
frame_no = 4
img_width = 64
img_height = 64
target_class = 7

<h4> Ravdess Video </h4>

In [ ]:
ravdess_actors = os.listdir('/kaggle/input/audio-visual/data/Video Speech Actor')
ravdess_actors.sort()
video_path = '/kaggle/input/audio-visual/data/Video Speech Actor'

for actor in ravdess_actors:
    folder_path = os.path.join(video_path, actor)
    videos = os.listdir(folder_path)
    for video in videos:
        p2 = os.path.join(folder_path, video)
        cam = cv2.VideoCapture(p2)
        fps = cam.get(cv2.CAP_PROP_FPS)
        frame_index = np.round(np.linspace(np.round(1*fps), np.round(2.3*fps), frame_no, dtype=np.int8))
        counter = 0
        while True:
            success, frame = cam.read(cv2.IMREAD_COLOR)
            if not success:
                break
            if counter in frame_index:
                frame = cv2.resize(frame, (img_width, img_height), cv2.INTER_AREA)
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                video_data.append((frame/255.0).astype('float32'))
            counter = counter+1
        cam.release()

<h4> Savee Video </h4>

In [ ]:
savee_video_path = '/kaggle/input/savee-dataset/video/video'
files = os.listdir(savee_video_path)

for file in files:
    p1 = os.path.join(savee_video_path, file)
    cam = cv2.VideoCapture(p1)
    fps = cam.get(cv2.CAP_PROP_FPS)
    frame_index = np.round(np.linspace(np.round(0.5*fps), np.round(1.5*fps), frame_no, dtype=np.int8))
    counter = 0
    while True:
        success, frame = cam.read(cv2.IMREAD_COLOR)
        if not success:
            break
        if counter in frame_index:
            frame = cv2.resize(frame, (img_width, img_height), cv2.INTER_AREA)
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            video_data.append((frame/255.0).astype('float32'))
        counter = counter+1
    cam.release()

In [ ]:
video_data = np.array(video_data).reshape(-1, frame_no, img_width, img_height, 3)
video_data.shape

<h3> Audio Data Reading </h3>

In [ ]:
MFCC = []
MEL_SPEC = []
CONTRAST = []
TONE = []
Labels = []

<h4> Ravdess Audio </h4>

In [ ]:
emotion = {
    '01':'neutral',
    '02':'neutral',
    '03':'happy',
    '04':'sad',
    '05':'angry',
    '06':'fearful',
    '07':'disgust',
    '08':'surprised'
}

In [ ]:
def get_gender(x):
    num = int(x.split('.')[0])
    if num%2 == 0:
        return 'female_'
    else:
        return 'male_'

In [ ]:
audio_actors = os.listdir('/kaggle/input/audio-visual/data/RAVDESS')
audio_actors.sort()

audio_path = '/kaggle/input/audio-visual/data/RAVDESS'

for actor in audio_actors:
    folder_path = os.path.join(audio_path, actor)
    audios = os.listdir(folder_path)
    for audio in audios:
        path = os.path.join(folder_path, audio)
        ad, sampling = librosa.load(path, sr = 22050*2, offset=0.5, duration=1.7, res_type='kaiser_fast')
        mfcc = np.mean(librosa.feature.mfcc(y = ad, sr=sampling, n_mfcc=10), axis=0).tolist()
        mel_spec = np.mean(librosa.feature.melspectrogram(y = ad, sr=sampling, n_mels=10), axis=0).tolist()
#         contrast = np.mean(librosa.feature.spectral_contrast(y = ad, sr=sampling, n_bands=6), axis=0).tolist()
#         tone = np.mean(librosa.feature.tonnetz(y = ad, sr=sampling), axis=0).tolist()
        MFCC.append(mfcc)
        MEL_SPEC.append(mel_spec)
#         CONTRAST.append(contrast)
#         TONE.append(tone)
        lb = audio.split('-')
        label = emotion[lb[2]]
        Labels.append(label)

<h4> Savee Audio </h4>

In [ ]:
def get_emotion(x):
    if 'sa' in x:
        return 'sad'
    elif 'su' in x:
        return 'surprised'
    elif 'a' in x:
        return 'angry'
    elif 'd' in x:
        return 'disgust'
    elif 'f' in x:
        return 'fearful'
    elif 'h' in x:
        return 'happy'
    elif 'n' in x:
        return 'neutral'

In [ ]:
savee_audio_path = '/kaggle/input/savee-dataset/audio/audio'
files = os.listdir(savee_audio_path)

for file in files:
    p1 = os.path.join(savee_audio_path, file)
    ad, sampling = librosa.load(p1, sr=22050*2, offset=0.5, duration=1.7, res_type='kaiser_fast')
    mfcc = np.mean(librosa.feature.mfcc(y = ad, sr=sampling, n_mfcc=10), axis=0).tolist()
    mel_spec = np.mean(librosa.feature.melspectrogram(y = ad, sr=sampling, n_mels=10), axis=0).tolist()
#     contrast = np.mean(librosa.feature.spectral_contrast(y=ad, sr=sampling, n_bands=6), axis=0).tolist()
#     tone = np.mean(librosa.feature.tonnetz(y = ad, sr=sampling), axis=0).tolist()
    MFCC.append(mfcc)
    MEL_SPEC.append(mel_spec)
#     CONTRAST.append(contrast)
#     TONE.append(tone)
    label = get_emotion(file.split('.')[0])
    Labels.append(label)

In [ ]:
df1 = pd.DataFrame(MFCC)
df2 = pd.DataFrame(MEL_SPEC)
# df3 = pd.DataFrame(CONTRAST)
# df4 = pd.DataFrame(TONE)

print(df1.shape)
print(df2.shape)
# print(df3.shape)
# print(df4.shape)

df1.fillna(0.0, inplace=True)
df2.fillna(0.0, inplace=True)
# df3.fillna(0.0, inplace=True)
# df4.fillna(0.0, inplace=True)

In [ ]:
audio_data = pd.concat([df1, df2], axis=1)
audio_data.columns = np.arange(0, audio_data.shape[-1])
audio_data.shape

In [ ]:
audio_data
# df1

In [ ]:
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# from sklearn.metrics import confusion_matrix
from tensorflow.keras import regularizers

In [ ]:
le = LabelEncoder()
Labels = le.fit_transform(Labels)

In [ ]:
le.classes_

In [ ]:
video_train, video_test, audio_train, audio_test, y_train, y_test = train_test_split(video_data, audio_data, Labels, test_size=0.25, shuffle=True)
# video_train, video_test, audio_train, audio_test, y_train, y_test = train_test_split(video_data, df1, Labels, test_size=0.3, shuffle=True)

<h3> Video Feature Extractor Model</h3>

In [ ]:
video_model = keras.Sequential()

video_model.add(keras.Input(shape=(frame_no, img_width, img_height, 3)))
video_model.add(layers.ConvLSTM2D(64, (3, 3), activation='relu'))
video_model.add(layers.Conv2D(64, (3, 3), activation='relu'))
video_model.add(layers.BatchNormalization())
video_model.add(layers.MaxPooling2D((2, 2)))
video_model.add(layers.Dropout(0.2))

video_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
video_model.add(layers.Conv2D(128, (3, 3), activation='relu'))
video_model.add(layers.BatchNormalization())
video_model.add(layers.MaxPooling2D((2, 2)))
video_model.add(layers.Dropout(0.2))


video_model.add(layers.Conv2D(256, (3, 3), activation='relu'))
video_model.add(layers.Conv2D(256, (3, 3), activation='relu'))
video_model.add(layers.BatchNormalization())
video_model.add(layers.MaxPooling2D((2, 2)))
video_model.add(layers.Dropout(0.2))

video_model.add(layers.Conv2D(512, (3, 3), activation='relu'))
video_model.add(layers.Conv2D(512, (3, 3), activation='relu'))
video_model.add(layers.BatchNormalization())
video_model.add(layers.MaxPooling2D((2, 2)))
video_model.add(layers.Dropout(0.2))

# video_model.add(layers.GlobalAveragePooling2D())

video_model.add(layers.Flatten())
video_model.summary()

<h3> Audio Feature Extractor Model</h3>

In [ ]:
audio_model = keras.Sequential()

audio_model.add(layers.Input(shape=(audio_data.shape[-1], 1)))
audio_model.add(layers.Conv1D(64, 10, activation='relu'))
audio_model.add(layers.Conv1D(64, 10, activation='relu'))
audio_model.add(layers.BatchNormalization())
audio_model.add(layers.MaxPooling1D(4))
audio_model.add(layers.Dropout(0.2))


audio_model.add(layers.Conv1D(128, 10, activation='relu'))
audio_model.add(layers.Conv1D(128, 10, activation='relu'))
audio_model.add(layers.BatchNormalization())
audio_model.add(layers.MaxPooling1D(4))
audio_model.add(layers.Dropout(0.2))

# audio_model.add(layers.Flatten())
# audio_model.summary()

# audio_model.add(layers.Input(shape=(audio_data.shape[-1], 1)))
# audio_model.add(layers.Conv1D(256, 8, activation='relu'))
# audio_model.add(layers.Conv1D(256, 8, activation='relu'))
# audio_model.add(layers.BatchNormalization())
# audio_model.add(layers.MaxPool1D(pool_size=2))
# audio_model.add(layers.Dropout(0.2))

# audio_model.add(layers.Conv1D(128, 8, activation='relu'))
# audio_model.add(layers.Conv1D(128, 8, activation='relu'))
# audio_model.add(layers.BatchNormalization())
# audio_model.add(layers.MaxPool1D(pool_size=2))
# audio_model.add(layers.Dropout(0.2))
# audio_model.add(layers.Conv1D(64, 8, activation='relu'))
# audio_model.add(layers.Conv1D(64, 8, activation='relu'))    
# audio_model.add(layers.BatchNormalization())
# audio_model.add(layers.MaxPooling1D(pool_size=2))
# audio_model.add(layers.Dropout(0.2))

audio_model.add(layers.Flatten())
audio_model.summary()

<h3> Model Fusion </h3>

In [ ]:
video_input = video_model.input
video_output = video_model.output
audio_input = audio_model.input
audio_output = audio_model.output


merge = layers.Concatenate()([video_output, audio_output])
out = layers.Dense(256, activation='relu', kernel_regularizer=regularizers.L2(0.01))(merge)
out = layers.Dropout(0.2)(out)
out = layers.Dense(128, activation='relu', kernel_regularizer=regularizers.L2(0.01))(out)
out = layers.Dropout(0.2)(out)
out = layers.Dense(target_class, activation='softmax')(out)

model = Model(inputs=[video_input, audio_input], outputs=out)
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model_check_point = tf.keras.callbacks.ModelCheckpoint(
    filepath = '/kaggle/working/',
    monitor = 'val_accuracy',
    mode='max',
    save_best_only = True
)

In [ ]:
history = model.fit([video_train, audio_train], y_train, validation_data=([video_test, audio_test], y_test), shuffle=True,
                    batch_size = 20, callbacks=[model_check_point], epochs=60)

In [ ]:
import matplotlib.pyplot as plt
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'y', label='Training loss')
plt.plot(epochs, val_loss, 'r', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.plot(epochs, acc, 'y', label='Training acc')
plt.plot(epochs, val_acc, 'r', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

<h4> Data Visualization </h4>

In [ ]:
path = '/kaggle/input/audio-visual/data/RAVDESS/Actor_01/03-01-02-01-02-02-01.wav'
audio, sr = librosa.load(path, sr=22050, offset=0.5, duration=2, res_type='kaiser_fast')

mfcc = np.mean(librosa.feature.mfcc(y=audio, sr=sr, n_mfcc=10), axis=0)
mel_spec = np.mean(librosa.feature.melspectrogram(y=audio, sr=sr, n_mels=10), axis=0)
contrast = np.mean(librosa.feature.spectral_contrast(y=audio, sr=sr), axis=0)
tone = np.mean(librosa.feature.tonnetz(y=audio, sr=sr), axis=0)

In [ ]:
plt.figure(figsize=(20, 15))
plt.subplot(411)
librosa.display.specshow(mfcc)
plt.subplot(412)
librosa.display.specshow(mel_spec)
plt.subplot(413)
librosa.display.specshow(contrast)
plt.subplot(414)
librosa.display.specshow(tone)
plt.colorbar()

In [ ]:
plt.figure(figsize=(15, 10))
plt.subplot(221)
plt.plot(np.mean(mfcc, axis=0))
plt.title('mfcc')
plt.subplot(222)
plt.plot(np.mean(mel_spec, axis=0))
plt.title('melspectrogram')
plt.subplot(223)
plt.plot(np.mean(contrast, axis=0))
plt.title('spectral contrat')
plt.subplot(224)
plt.plot(np.mean(tone, axis=0))
plt.title('tonnetz')
